In [23]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import glob
import numpy as np
import tqdm
import datetime
import geopandas as gpd
import rtree

In [4]:
files = glob.glob("data/raw/SNData/*.csv")
#dfs = [pd.read_csv(f, header=0, sep=";", parse_dates = [7,8]) for f in files]

dfs = []
for f in tqdm.tqdm(files):
    dfs.append(pd.read_csv(f, header=0, sep=";", parse_dates = [7,8]))

Full_data = pd.concat(dfs,ignore_index=True) # Save this to interim
Full_data.to_csv('data/interim/Full_data.csv')

100%|██████████| 53/53 [07:58<00:00,  9.03s/it]


In [5]:
df = Full_data.dropna() # drops 53 values
df. columns = ['Customer_Group', 'CustomerID', 'CarID', 'Engine', 'Rental_flag', 'RentalID', 'Rental_Usage_Type', 'Reservation_Time', 'End_Time', 'Revenue', 'Distance', 'Drives', 'Reservation_Minutes','Fuel_Start','Fuel_End','Start_Lat', 'Start_Long', 'End_Lat', 'End_Long']
df = df.drop(columns = 'Drives') # No info (only ones)
df

,Customer_Group,CustomerID,CarID,Engine,Rental_flag,RentalID,Rental_Usage_Type,Reservation_Time,End_Time,Revenue,Distance,Reservation_Minutes,Fuel_Start,Fuel_End,Start_Lat,Start_Long,End_Lat,End_Long
0,Non_Customer,793639.0,WBY1Z21080V307924,I3,No,9.335872e+09,Private,2016-03-24 11:48:43,2016-02-04 10:00:19,0.00,0,0,0,0,55.678763,12.552853,0.000000,0.000000
1,Non_Customer,1035973.0,WBY1Z21080V307857,I3,No,9.336114e+09,Private,2016-03-30 15:37:39,2016-01-04 00:40:38,0.00,0,0,62,47,55.770626,12.519300,55.770389,12.518839
2,Non_Customer,998095.0,WBY1Z21020V307904,I3,No,9.336154e+09,Private,2016-03-31 13:08:16,2016-05-04 08:32:25,0.00,2,1,85,79,55.621588,12.606951,55.621532,12.606279
3,Non_Customer,999604.0,WBY1Z21010V307926,I3,No,9.336158e+09,Private,2016-03-31 14:43:00,2016-01-04 07:10:00,0.00,0,1,0,71,55.770077,12.518914,55.769746,12.519123
4,Non_Customer,1035969.0,WBY1Z21070V308210,,No,9.336160e+09,Private,2016-03-31 15:21:36,2016-01-04 14:24:17,0.00,0,1,53,52,55.770623,12.519791,55.770439,12.518937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2633979,Customer,1070662.0,WBY1Z21010V308185,I3,No,9.345011e+09,Private,2016-09-30 23:39:05,2016-09-30 23:50:54,5.16,4,10,46,41,55.694700,12.553776,55.678740,12.587144
2633980,Customer,1041705.0,WBY1Z21080V308250,I3,No,9.345011e+09,Private,2016-09-30 23:42:18,2016-09-30 23:52:14,3.44,6,8,59,52,55.648401,12.542945,55.641310,12.615295
2633981,Customer,2112471.0,WBY1Z21020V308261,I3,No,9.345011e+09,Private,2016-09-30 23:33:39,2016-09-30 23:52:03,8.17,9,3,39,30,55.664744,12.580875,55.719856,12.540863
2633982,Customer,440147.0,WBY1Z21060V307954,I3,Yes,9.345011e+09,Private,2016-09-30 23:41:56,2016-09-30 23:57:30,6.88,9,4,44,35,55.710676,12.566043,55.667453,12.619987


In [6]:
df = df[df.CarID != '0']

In [7]:
df["Engine"].replace({" ": '0'}, inplace=True) # Ensure the two "missing" are aligned

In [8]:
Engine_dict = {c: df[df.CarID == c].Engine.nunique() for c in df[df.Engine == '0'].CarID.unique()}
for car, engine in Engine_dict.items():
    if engine == 1:
        continue
    True_Engine = [x for x in df[df.CarID == car].Engine.unique() if x!= '0'][0]
    df.loc[(df.CarID == car) & (df.Engine == '0'), 'Engine'] = True_Engine

# Populate the rest manual based on ID
df.loc[(df.CarID == 'WBA1R5104J7B14310') & (df.Engine == '0'), 'Engine'] = '118I'
df.loc[(df.CarID == 'WBA1R5104J5K58061') & (df.Engine == '0'), 'Engine'] = '118I'
df.loc[(df.CarID == 'WBA1R5103K7D66678') & (df.Engine == '0'), 'Engine'] = '118I'
df.loc[(df.CarID == 'WBY8P2105K7D70350') & (df.Engine == '0'), 'Engine'] = 'I3 120'
df.loc[(df.CarID == 'WBY8P2102K7D70287') & (df.Engine == '0'), 'Engine'] = 'I3 120'

In [9]:
df['Start_Time'] = [row.Reservation_Time+datetime.timedelta(minutes=row.Reservation_Minutes) for _, row in df.iterrows()] 

,Customer_Group,CustomerID,CarID,Engine,Rental_flag,RentalID,Rental_Usage_Type,Reservation_Time,End_Time,Revenue,Distance,Reservation_Minutes,Fuel_Start,Fuel_End,Start_Lat,Start_Long,End_Lat,End_Long,Start_Time
0,Non_Customer,793639.0,WBY1Z21080V307924,I3,No,9.335872e+09,Private,2016-03-24 11:48:43,2016-02-04 10:00:19,0.00,0,0,0,0,55.678763,12.552853,0.000000,0.000000,2016-03-24 11:48:43
1,Non_Customer,1035973.0,WBY1Z21080V307857,I3,No,9.336114e+09,Private,2016-03-30 15:37:39,2016-01-04 00:40:38,0.00,0,0,62,47,55.770626,12.519300,55.770389,12.518839,2016-03-30 15:37:39
2,Non_Customer,998095.0,WBY1Z21020V307904,I3,No,9.336154e+09,Private,2016-03-31 13:08:16,2016-05-04 08:32:25,0.00,2,1,85,79,55.621588,12.606951,55.621532,12.606279,2016-03-31 13:09:16
3,Non_Customer,999604.0,WBY1Z21010V307926,I3,No,9.336158e+09,Private,2016-03-31 14:43:00,2016-01-04 07:10:00,0.00,0,1,0,71,55.770077,12.518914,55.769746,12.519123,2016-03-31 14:44:00
4,Non_Customer,1035969.0,WBY1Z21070V308210,I3,No,9.336160e+09,Private,2016-03-31 15:21:36,2016-01-04 14:24:17,0.00,0,1,53,52,55.770623,12.519791,55.770439,12.518937,2016-03-31 15:22:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2633979,Customer,1070662.0,WBY1Z21010V308185,I3,No,9.345011e+09,Private,2016-09-30 23:39:05,2016-09-30 23:50:54,5.16,4,10,46,41,55.694700,12.553776,55.678740,12.587144,2016-09-30 23:49:05
2633980,Customer,1041705.0,WBY1Z21080V308250,I3,No,9.345011e+09,Private,2016-09-30 23:42:18,2016-09-30 23:52:14,3.44,6,8,59,52,55.648401,12.542945,55.641310,12.615295,2016-09-30 23:50:18
2633981,Customer,2112471.0,WBY1Z21020V308261,I3,No,9.345011e+09,Private,2016-09-30 23:33:39,2016-09-30 23:52:03,8.17,9,3,39,30,55.664744,12.580875,55.719856,12.540863,2016-09-30 23:36:39
2633982,Customer,440147.0,WBY1Z21060V307954,I3,Yes,9.345011e+09,Private,2016-09-30 23:41:56,2016-09-30 23:57:30,6.88,9,4,44,35,55.710676,12.566043,55.667453,12.619987,2016-09-30 23:45:56


In [32]:
tmp = df[df.End_Time < df.Reservation_Time]
tmp.sort_values(by='Reservation_Time')

,Customer_Group,CustomerID,CarID,Engine,Rental_flag,RentalID,Rental_Usage_Type,Reservation_Time,End_Time,Revenue,...,Reservation_Minutes,Fuel_Start,Fuel_End,Start_Lat,Start_Long,End_Lat,End_Long,Start_Time,Start_Zone,End_Zone
706796,Non_Customer,793639.0,WBY1Z21010V307859,I3,No,9.328461e+09,Private,2015-02-10 17:39:46,2015-01-10 20:24:48,0.00,...,1892,99,87,55.652858,12.613396,55.675784,12.559089,2015-02-12 01:11:46,103222,102162
710614,Customer,804386.0,WBY1Z21090V308029,I3,No,9.328694e+09,Private,2015-07-10 19:31:18,2015-07-10 07:40:17,0.00,...,1242,99,96,55.676259,12.569459,55.682856,12.610024,2015-07-11 16:13:18,102131,103133
1596912,Non_Customer,819723.0,WBY1Z21000V307870,I3,No,9.327687e+09,Private,2015-09-15 08:29:39,2015-09-15 08:17:29,0.00,...,4070,62,56,55.675709,12.543872,55.696982,12.528448,2015-09-18 04:19:39,147131,102542
1597105,Non_Customer,819723.0,WBY1Z21000V307870,I3,No,9.327709e+09,Private,2015-09-15 15:28:32,2015-09-15 15:23:47,0.00,...,359,51,49,55.674612,12.560151,55.676148,12.559185,2015-09-15 21:27:32,102162,102162
1598329,Non_Customer,793639.0,WBY1Z21080V308202,I3,No,9.327795e+09,Private,2015-09-17 12:07:09,2015-09-17 07:25:39,0.00,...,1219,82,82,55.666756,12.546134,55.666730,12.546229,2015-09-18 08:26:09,102821,102821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914783,Customer,3683899.0,WBY8P2107K7E73561,I3 120,No,9.428659e+09,Private,2019-12-11 23:54:48,2019-11-13 00:17:23,2.14,...,15,83,79,55.645829,12.633539,55.641120,12.599504,2019-12-12 00:09:48,103211,103252
1914962,Customer,2487732.0,WBY8P2108K7E74377,I3 120,No,9.428662e+09,Private,2019-12-11 23:54:56,2019-11-13 06:57:37,8.03,...,409,26,19,55.648879,12.551198,55.630202,12.648771,2019-12-12 06:43:56,102851,185203
1914776,Customer,3689127.0,WBY8P2107K7D75453,I3 120,No,9.428658e+09,Private,2019-12-11 23:56:00,2019-11-13 00:12:54,3.85,...,9,28,25,55.682343,12.551068,55.689964,12.560512,2019-12-12 00:05:00,147112,102442
1914758,Customer,3592540.0,WBY8P2102K7D75442,I3 120,No,9.428658e+09,Private,2019-12-11 23:56:52,2019-11-13 00:13:13,5.14,...,5,42,38,55.685301,12.567428,55.684983,12.484528,2019-12-12 00:01:52,102151,102632


In [33]:
tmp.CarID.value_counts()

WBY1Z21060V308263    17
WBY1Z21060V308246    16
WBA1R5101J5K58065    16
WBY1Z21050V308285    15
WBY1Z21060V308053    15
                     ..
WBY8P2102K7E71720     1
WBY8P2101K7D73536     1
WBY8P2101K7E72616     1
WBY8P2100K7D75665     1
WBY1Z21080V307924     1
Name: CarID, Length: 894, dtype: int64

## Add zones

In [3]:
#df = pd.read_csv('data/processed/tmpdf.csv',index_col =0)

In [24]:
shapefile = gpd.read_file("../Zonekort/LTM_Zone3/zones_level3.shp")
shapefile = shapefile.to_crs(epsg=4326)

In [25]:
gdf_start = gpd.GeoDataFrame(df, geometry= gpd.points_from_xy(df.Start_Long, df.Start_Lat))

# Set projection
gdf_start = gdf_start.set_crs(epsg=4326)

In [26]:
gdpj_start  = gpd.sjoin(gdf_start, shapefile, op='within')
df['Start_Zone'] = gdpj_start.zoneid

In [27]:
Start_zone_filler = {x: shapefile.zoneid[shapefile.distance(df.loc[x].geometry).sort_values().index[0]] for x in df.index[df['Start_Zone'].isna()]}
df['Start_Zone'] = df['Start_Zone'].fillna(Start_zone_filler)

/var/folders/3z/pggbcw7949507j0lbw1gjngc0000gn/T/ipykernel_70464/1471600849.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  Start_zone_filler = {x: shapefile.zoneid[shapefile.distance(df.loc[x].geometry).sort_values().index[0]] for x in df.index[df['Start_Zone'].isna()]}


In [28]:
gdf_end = gpd.GeoDataFrame(df, geometry= gpd.points_from_xy(df.End_Long, df.End_Lat))

# Set projection
gdf_end = gdf_end.set_crs(epsg=4326)

In [29]:
gdpj_end  = gpd.sjoin(gdf_end, shapefile, op='within')
df['End_Zone'] = gdpj_end.zoneid

In [30]:
End_zone_filler = {x: shapefile.zoneid[shapefile.distance(df.loc[x].geometry).sort_values().index[0]] for x in df.index[df['End_Zone'].isna()]}
df['End_Zone'] = df['End_Zone'].fillna(End_zone_filler)

/var/folders/3z/pggbcw7949507j0lbw1gjngc0000gn/T/ipykernel_70464/2411542197.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  End_zone_filler = {x: shapefile.zoneid[shapefile.distance(df.loc[x].geometry).sort_values().index[0]] for x in df.index[df['End_Zone'].isna()]}


In [37]:
df = df.drop(columns = 'geometry')
df = df.astype({'CustomerID': 'int32', 'RentalID': 'int64', 'Start_Zone': 'int32','End_Zone': 'int32'})

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2630477 entries, 0 to 2633983
Data columns (total 21 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Customer_Group       object        
 1   CustomerID           int32         
 2   CarID                object        
 3   Engine               object        
 4   Rental_flag          object        
 5   RentalID             int64         
 6   Rental_Usage_Type    object        
 7   Reservation_Time     datetime64[ns]
 8   End_Time             datetime64[ns]
 9   Revenue              float64       
 10  Distance             int64         
 11  Reservation_Minutes  int64         
 12  Fuel_Start           int64         
 13  Fuel_End             int64         
 14  Start_Lat            float64       
 15  Start_Long           float64       
 16  End_Lat              float64       
 17  End_Long             float64       
 18  Start_Time           datetime64[ns]
 19  Start_Zone           

## Weird Coordinates

In [42]:
# Equator 1
df[(df.Start_Lat < 5)]

,Customer_Group,CustomerID,CarID,Engine,Rental_flag,RentalID,Rental_Usage_Type,Reservation_Time,End_Time,Revenue,...,Reservation_Minutes,Fuel_Start,Fuel_End,Start_Lat,Start_Long,End_Lat,End_Long,Start_Time,Start_Zone,End_Zone
318973,Non_Customer,793661,WBY1Z6108HV938983,I3 94,No,9349129851,Private,2016-12-22 15:38:31,2016-12-22 15:57:30,0.0,...,1,86,84,0.0,0.0,55.703077,12.553715,2016-12-22 15:39:31,550062,102432
589405,Non_Customer,793639,WBY8P2107K7E71728,I3 120,No,9422076259,Private,2019-09-19 14:29:27,2019-09-19 15:08:06,0.0,...,6,9,68,0.0,0.0,55.674228,12.560673,2019-09-19 14:35:27,550062,102161
592152,Non_Customer,793639,WBA31AA02L5N90396,X1 SDRIVE18I,No,9422140700,Private,2019-09-20 12:06:08,2019-09-20 12:22:58,0.0,...,15,0,0,0.0,0.0,55.716238,12.567008,2019-09-20 12:21:08,550062,102412
607119,Non_Customer,793639,WBY8P2104K7E72500,I3 120,No,9422802637,Private,2019-09-25 16:57:17,2019-09-25 17:23:20,0.0,...,1,7,6,0.0,0.0,55.716530,12.566844,2019-09-25 16:58:17,550062,102412
609129,Non_Customer,793639,WBY8P2106K7E72613,I3 120,No,9422919959,Private,2019-09-26 11:51:09,2019-09-26 11:58:08,0.0,...,3,5,100,0.0,0.0,55.715922,12.575446,2019-09-26 11:54:09,550062,102324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571451,Non_Customer,1106037,WMWXU7107KTM91384,COOPER,No,9379438620,Private,2018-06-15 06:01:07,2018-06-15 06:06:06,0.0,...,3,99,99,0.0,0.0,55.640906,12.611920,2018-06-15 06:04:07,550062,103251
2573547,Non_Customer,1114084,WMWXU7101KTM91395,COOPER,No,9379504274,Private,2018-06-15 21:18:48,2018-06-15 21:28:11,0.0,...,2,97,90,0.0,0.0,55.630224,12.648739,2018-06-15 21:20:48,550062,185203
2587868,Non_Customer,793639,WMWXR3100KTK19883,COOPER,No,9379880464,Private,2018-06-21 18:38:45,2018-06-21 19:47:16,0.0,...,28,74,72,0.0,0.0,55.674168,12.560709,2018-06-21 19:06:45,550062,102161
2589662,Non_Customer,819011,WMWXU7107KTM90591,COOPER,No,9379929925,Private,2018-06-22 12:52:55,2018-06-22 13:13:24,0.0,...,2,90,87,0.0,0.0,55.622541,12.615536,2018-06-22 12:54:55,550062,185154


In [43]:
# Equator 2
df[df.End_Lat < 5]

,Customer_Group,CustomerID,CarID,Engine,Rental_flag,RentalID,Rental_Usage_Type,Reservation_Time,End_Time,Revenue,...,Reservation_Minutes,Fuel_Start,Fuel_End,Start_Lat,Start_Long,End_Lat,End_Long,Start_Time,Start_Zone,End_Zone
0,Non_Customer,793639,WBY1Z21080V307924,I3,No,9335872135,Private,2016-03-24 11:48:43,2016-02-04 10:00:19,0.00,...,0,0,0,55.678763,12.552853,0.0,0.0,2016-03-24 11:48:43,147111,550062
1646,Customer,1030857,WBY1Z21050V307783,I3,No,9336279892,Private,2016-02-04 23:28:40,2016-03-04 03:06:32,78.57,...,15,44,0,55.667531,12.549165,0.0,0.0,2016-02-04 23:43:40,102821,550062
4685,Customer,819446,WBY1Z21050V308108,I3,No,9336476091,Private,2016-07-04 13:41:24,2016-07-04 14:27:27,20.21,...,17,100,0,55.629616,12.580891,0.0,0.0,2016-07-04 13:58:24,103291,550062
14282,Customer,1069340,WBY1Z21010V307859,I3,No,9337055461,Private,2016-04-19 21:11:17,2016-04-19 22:14:48,22.15,...,3,31,0,55.674121,12.499912,0.0,0.0,2016-04-19 21:14:17,147251,550062
19084,Customer,796025,WBY1Z21010V308073,I3,No,9337342253,Private,2016-04-26 01:39:19,2016-04-26 02:08:26,0.00,...,11,39,96,55.674215,12.560683,0.0,0.0,2016-04-26 01:50:19,102161,550062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2627413,Customer,800699,WBY1Z21090V307804,I3,No,9344591671,Private,2016-09-22 15:42:22,2016-09-22 16:07:30,9.01,...,8,58,0,55.671578,12.574404,0.0,0.0,2016-09-22 15:50:22,102171,550062
2629163,Customer,820349,WBY1Z21000V307917,I3,No,9344701319,Private,2016-09-24 17:52:07,2016-09-24 20:38:13,45.07,...,0,75,0,55.696387,12.594131,0.0,0.0,2016-09-24 17:52:07,102336,550062
2631059,Customer,2043660,WBY1Z21070V307929,I3,No,9344825951,Private,2016-09-27 13:41:38,2016-09-27 14:59:46,28.56,...,5,85,0,55.665693,12.621481,0.0,0.0,2016-09-27 13:46:38,103151,550062
2631542,Customer,2118489,WBY1Z21090V307821,I3,No,9344857319,Private,2016-09-28 08:51:39,2016-09-28 09:39:17,27.16,...,2,33,0,55.630176,12.648790,0.0,0.0,2016-09-28 08:53:39,185203,550062


In [44]:
# Sweden and Bornholm
df[df.Start_Long > 13]

,Customer_Group,CustomerID,CarID,Engine,Rental_flag,RentalID,Rental_Usage_Type,Reservation_Time,End_Time,Revenue,...,Reservation_Minutes,Fuel_Start,Fuel_End,Start_Lat,Start_Long,End_Lat,End_Long,Start_Time,Start_Zone,End_Zone
351052,Customer,866569,WBA2C1106GV519875,218D,No,9335030622,Private,2016-05-03 13:45:22,2016-10-03 20:10:57,230.41,...,32,18,27,59.332089,18.051437,58.111483,12.150692,2016-05-03 14:17:22,400114,825001
639391,Non_Customer,809751,WBY1Z21080V308216,I3,No,9357885653,Private,2017-06-13 10:14:58,2017-06-13 10:17:44,0.00,...,1,0,0,55.101044,14.685184,55.069769,14.750558,2017-06-13 10:15:58,400334,400147
639428,Non_Customer,809751,WBY1Z21050V308111,I3,No,9357888002,Private,2017-06-13 10:55:28,2017-06-13 10:56:42,0.00,...,1,0,0,55.101614,14.685244,55.069851,14.750575,2017-06-13 10:56:28,400334,400147
639460,Non_Customer,809751,WBY1Z21020V307871,I3,No,9357890020,Private,2017-06-13 11:30:21,2017-06-13 11:33:12,0.00,...,1,0,0,55.069446,14.750443,55.069788,14.751023,2017-06-13 11:31:21,400147,400147
639528,Non_Customer,809751,WBY1Z21030V308138,I3,No,9357893867,Private,2017-06-13 12:48:07,2017-06-13 12:49:17,0.00,...,1,0,0,55.100947,14.685519,55.102003,14.690333,2017-06-13 12:49:07,400334,400334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657313,Non_Customer,793639,WBY1Z21010V307876,I3,No,9358745431,Private,2017-06-28 11:14:30,2017-06-28 12:01:40,0.00,...,42,8,11,55.100701,14.686954,55.100698,14.686948,2017-06-28 11:56:30,400334,400334
733460,Customer,829813,WBY1Z21020V307840,I3,No,9331420174,Private,2015-08-12 22:04:01,2015-08-12 23:03:44,20.37,...,5,34,4,55.606053,13.025068,55.602430,12.660331,2015-08-12 22:09:01,185121,155013
766566,Non_Customer,793661,WBY1Z210X0V307892,I3,No,9354880509,Private,2017-04-19 10:49:33,2017-04-19 13:17:08,0.00,...,2,32,72,55.276373,14.802931,55.277839,14.802453,2017-04-19 10:51:33,400213,400213
766664,Non_Customer,793661,WBY1Z210X0V307892,I3,No,9354888567,Private,2017-04-19 13:20:16,2017-04-19 13:49:04,0.00,...,2,0,0,55.277839,14.802453,55.278269,14.799411,2017-04-19 13:22:16,400213,400213


In [45]:
# Jutland
df[df.Start_Long < 11]

,Customer_Group,CustomerID,CarID,Engine,Rental_flag,RentalID,Rental_Usage_Type,Reservation_Time,End_Time,Revenue,...,Reservation_Minutes,Fuel_Start,Fuel_End,Start_Lat,Start_Long,End_Lat,End_Long,Start_Time,Start_Zone,End_Zone
318973,Non_Customer,793661,WBY1Z6108HV938983,I3 94,No,9349129851,Private,2016-12-22 15:38:31,2016-12-22 15:57:30,0.0,...,1,86,84,0.000000,0.000000,55.703077,12.553715,2016-12-22 15:39:31,550062,102432
496868,Non_Customer,793639,WMWXU7105KTM91318,COOPER,No,9419190956,Private,2019-08-19 08:04:36,2019-08-19 08:47:12,0.0,...,20,5,100,56.461193,10.035505,55.683235,12.585613,2019-08-19 08:24:36,730323,102223
589405,Non_Customer,793639,WBY8P2107K7E71728,I3 120,No,9422076259,Private,2019-09-19 14:29:27,2019-09-19 15:08:06,0.0,...,6,9,68,0.000000,0.000000,55.674228,12.560673,2019-09-19 14:35:27,550062,102161
592152,Non_Customer,793639,WBA31AA02L5N90396,X1 SDRIVE18I,No,9422140700,Private,2019-09-20 12:06:08,2019-09-20 12:22:58,0.0,...,15,0,0,0.000000,0.000000,55.716238,12.567008,2019-09-20 12:21:08,550062,102412
607119,Non_Customer,793639,WBY8P2104K7E72500,I3 120,No,9422802637,Private,2019-09-25 16:57:17,2019-09-25 17:23:20,0.0,...,1,7,6,0.000000,0.000000,55.716530,12.566844,2019-09-25 16:58:17,550062,102412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587868,Non_Customer,793639,WMWXR3100KTK19883,COOPER,No,9379880464,Private,2018-06-21 18:38:45,2018-06-21 19:47:16,0.0,...,28,74,72,0.000000,0.000000,55.674168,12.560709,2018-06-21 19:06:45,550062,102161
2589662,Non_Customer,819011,WMWXU7107KTM90591,COOPER,No,9379929925,Private,2018-06-22 12:52:55,2018-06-22 13:13:24,0.0,...,2,90,87,0.000000,0.000000,55.622541,12.615536,2018-06-22 12:54:55,550062,185154
2599546,Non_Customer,793661,WMWXU7108KTM91409,COOPER,No,9380185309,Private,2018-06-26 11:56:23,2018-06-26 12:43:40,0.0,...,1,80,77,55.434576,9.295855,55.692034,12.484833,2018-06-26 11:57:23,621243,102641
2599762,Non_Customer,793661,WBA1R510XJ7B14313,118I,No,9380192313,Private,2018-06-26 13:52:31,2018-06-26 13:57:24,0.0,...,2,92,90,55.435023,9.296272,55.675903,12.553119,2018-06-26 13:54:31,621243,147111


In [ ]:
df[df.Revenue <= 0.01].sort_values(by = 'Start_Time')